In [4]:
import pandas as pd
from finvizfinance.screener.overview import Overview

def get_filtered_picks(countries=['USA'], limit_per_country=5000):
    all_stocks = []
    
    for country in countries:
        print(f"Fetching candidates for: {country}...")
        
        try:
            foverview = Overview()
            
            # --- STEP 1: Broaden the initial search ---
            filters_dict = {
                'Analyst Recom.': 'Strong Buy (1)',
                'Country': country,
                'Average Volume': 'Over 2M', # Added volume filter for liquidity
                'Market Cap.': '+Small (over $300mln)'
            }
            foverview.set_filter(filters_dict=filters_dict)
            
            # Get results
            df_results = foverview.screener_view()
            
            if not df_results.empty:
                df_results['Source_Country'] = country
                all_stocks.append(df_results)
            else:
                print(f"   No stocks found for {country}.")
                
        except Exception as e:
            print(f"   Error fetching {country}: {e}")

    if not all_stocks:
        print("\nNo stocks found from any country.")
        return pd.DataFrame()

    # Combine all results
    combined_df = pd.concat(all_stocks, ignore_index=True)
    
    # --- STEP 2: The "Option 2" Pandas Filter ---
    cols = combined_df.columns.tolist()
    recom_col = 'Recom' if 'Recom' in cols else 'Analyst Recom'
    
    if recom_col in cols:
        # Convert column to numbers
        combined_df[recom_col] = pd.to_numeric(combined_df[recom_col], errors='coerce')
        
        # Filter: keep anything <= 1.5 (Strong Buys)
        print(f"Filtering out 'Hold' or worse (Rating > 1.5)...")
        combined_df = combined_df[combined_df[recom_col] <= 1.0]
        
        # Sort by best rating
        combined_df = combined_df.sort_values(by=recom_col, ascending=True)

    # Clean up columns
    desired_cols = ['Ticker', 'Company', 'Sector', 'Price', 'Source_Country', recom_col]
    final_cols = [c for c in desired_cols if c in combined_df.columns]
    
    return combined_df[final_cols].head(limit_per_country * len(countries))

# --- Usage ---
top_picks = get_filtered_picks(['USA'])

if not top_picks.empty:
    # Ensure Price is numeric (Crucial for sorting in Data Viewer)
    top_picks['Price'] = pd.to_numeric(top_picks['Price'], errors='coerce')

    print(f"\nSuccess! Found {len(top_picks)} candidates.")
    print("Variable 'top_picks' is ready for the Data Viewer.")

else:
    print("DataFrame is empty.")

Fetching candidates for: USA...
[Info] loading page [###########################---] 10/11 
Success! Found 220 candidates.
Variable 'top_picks' is ready for the Data Viewer.


In [5]:
import pandas as pd
import yfinance as yf
import numpy as np
from scipy.stats import linregress

def calculate_z_score(info, financials, balance_sheet):
    """
    Calculates Altman Z-Score for Non-Financial Firms.
    Formula: 1.2A + 1.4B + 3.3C + 0.6D + 1.0E
    """
    try:
        # Data Extraction
        total_assets = balance_sheet.loc['Total Assets'].iloc[0]
        total_liab = balance_sheet.loc['Total Liabilities Net Minority Interest'].iloc[0]
        current_assets = balance_sheet.loc['Current Assets'].iloc[0]
        current_liab = balance_sheet.loc['Current Liabilities'].iloc[0]
        
        # Working Capital
        working_capital = current_assets - current_liab
        
        # Retained Earnings (Use 0 if missing/deficit)
        retained_earnings = balance_sheet.loc['Retained Earnings'].iloc[0] if 'Retained Earnings' in balance_sheet.index else 0
        
        # EBIT
        ebit = financials.loc['Ebit'].iloc[0] if 'Ebit' in financials.index else financials.loc['Operating Income'].iloc[0]
        
        # Market Value of Equity (Market Cap)
        market_cap = info.get('marketCap', 0)
        
        # Sales (Total Revenue)
        sales = financials.loc['Total Revenue'].iloc[0]

        # Ratios
        A = working_capital / total_assets
        B = retained_earnings / total_assets
        C = ebit / total_assets
        D = market_cap / total_liab
        E = sales / total_assets

        # Z-Score Calculation
        z_score = (1.2 * A) + (1.4 * B) + (3.3 * C) + (0.6 * D) + (1.0 * E)
        return round(z_score, 2)
    
    except Exception:
        return np.nan

def get_margin_trend(financials):
    """Calculates slope of Gross Margin over last 3 years (Safe for $0 Revenue)"""
    try:
        # Get last 3 years of data
        years = financials.columns[:3] 
        margins = []
        
        for date in years:
            rev = financials.loc['Total Revenue'][date]
            profit = financials.loc['Gross Profit'][date]
            
            # SAFEGUARD: Prevent division by zero
            if rev == 0 or np.isnan(rev):
                margins.append(0) # Treat as 0% margin
            else:
                margins.append(profit / rev)
            
        # Reverse list so it's [Year-3, Year-2, Year-1] (Chronological)
        margins = margins[::-1]
        
        # Calculate Slope
        slope, _, _, _, _ = linregress(range(len(margins)), margins)
        
        if slope > 0.005: return "Improving"
        elif slope < -0.005: return "Deteriorating"
        else: return "Stable"
        
    except Exception:
        return "N/A"

def bucket_stocks(ticker_df):
    """
    Takes your 'top_picks' DataFrame and adds Credit/Risk Buckets.
    """
    print(f"--- Performing Credit Risk Analysis on {len(ticker_df)} Stocks ---")
    
    results = []
    
    for ticker in ticker_df['Ticker']:
        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            
            # Skip Financials (Banks) for Z-Score (Formula doesn't apply)
            sector = info.get('sector', 'Unknown')
            if 'Financial' in sector:
                results.append({'Ticker': ticker, 'Bucket': 'Financial (Skipped)', 'Z-Score': np.nan})
                continue
            
            # Fetch Financial Statements
            fin = stock.financials
            bs = stock.balance_sheet
            
            if fin.empty or bs.empty:
                continue

            # --- METRIC CALCULATIONS ---
            z_score = calculate_z_score(info, fin, bs)
            margin_trend = get_margin_trend(fin)
            
            # --- BUCKET LOGIC ---
            bucket = "Unclassified"
            
            # Bucket A: The Fortress (Safe)
            if (z_score > 2.99) and (margin_trend == "Improving" or margin_trend == "Stable"):
                bucket = "Fortress (Safe)"
                
            # Bucket C: Distress (Value Trap)
            elif (z_score < 1.8) and (margin_trend == "Deteriorating"):
                bucket = "Distress (AVOID)"
            
            # Bucket B: Speculative (Hype)
            elif (z_score < 1.8) and (margin_trend == "Improving"):
                 bucket = "Moonshot (High Risk)"
            
            else:
                 bucket = "Middle of the Road"

            results.append({
                'Ticker': ticker,
                'Bucket': bucket,
                'Z-Score': z_score,
                'Margin_Trend': margin_trend,
                'Sector': sector
            })
            
        except Exception as e:
            print(f"Skipping {ticker}: {e}")
            
    # Merge results back into original DataFrame
    risk_df = pd.DataFrame(results)
    final_df = pd.merge(ticker_df, risk_df, on='Ticker', how='left')
    return final_df

In [6]:
# 1. Run your existing Funnel
top_picks = get_filtered_picks(['USA'])

if not top_picks.empty:
    # 2. NEW: Run the Risk Analysis on a smaller list
    shortlist = top_picks.head(20).copy() 
    analyzed_picks = bucket_stocks(shortlist)
    
    # --- FIX: DETECT CORRECT COLUMN NAME ---
    # Check if 'Recom' exists, if not, try 'Analyst Recom'
    if 'Recom' in analyzed_picks.columns:
        recom_col = 'Recom'
    elif 'Analyst Recom' in analyzed_picks.columns:
        recom_col = 'Analyst Recom'
    else:
        recom_col = None # Column is missing entirely

    # 3. Define columns to display
    cols = ['Ticker', 'Company', 'Bucket', 'Z-Score', 'Margin_Trend', 'Price']
    if recom_col:
        cols.append(recom_col)
    
    # Sort so "Fortress" stocks appear at the top
    # We use 'ignore_index=True' to avoid sorting errors if index is messy
    analyzed_picks.sort_values(by='Z-Score', ascending=False, inplace=True)
    
    print(f"--- Risk Report Generated for {len(analyzed_picks)} stocks ---")
    display(analyzed_picks[cols])
else:
    print("No stocks found.")

Fetching candidates for: USA...
--- Performing Credit Risk Analysis on 20 Stocks --- 10/11 
--- Risk Report Generated for 20 stocks ---


,Ticker,Company,Bucket,Z-Score,Margin_Trend,Price
14,ANET,Arista Networks Inc,Fortress (Safe),27.19,Improving,131.12
3,ADMA,Adma Biologics Inc,Fortress (Safe),21.68,Improving,19.58
0,ABAT,American Battery Technology Company,Middle of the Road,16.66,Deteriorating,3.97
12,AMPX,Amprius Technologies Inc,Fortress (Safe),11.65,Improving,9.35
8,ALGM,Allegro Microsystems Inc,Middle of the Road,6.78,Deteriorating,26.70
13,AMZN,Amazon.com Inc,Fortress (Safe),6.09,Improving,227.35
9,ALHC,Alignment Healthcare Inc,Middle of the Road,5.30,Deteriorating,20.42
15,ANNX,Annexon Inc,Middle of the Road,4.80,N/A,5.17
1,ABSI,Absci Corp,Middle of the Road,4.44,N/A,3.38
17,APLD,Applied Digital Corporation,Fortress (Safe),3.34,Improving,27.85


In [7]:
# filter for price range less than $5 a bit gambly
if not top_picks.empty:
    # Ensure the 'Price' column is numeric
    top_picks['Price'] = pd.to_numeric(top_picks['Price'], errors='coerce')

    # CORRECT WAY: Use '&' with parentheses
    cheap_picks = top_picks[(top_picks['Price'] < 5)]

    print(f"\nFound {len(cheap_picks)} stocks less than $5")
    # print(cheap_picks) # Uncomment if you want to see the list in output
else:
    print("DataFrame is empty.")


Found 31 stocks less than $5


In [8]:
# filter for price range less than $5 a bit gambly
if not top_picks.empty:
    # Ensure the 'Price' column is numeric
    top_picks['Price'] = pd.to_numeric(top_picks['Price'], errors='coerce')

    # CORRECT WAY: Use '&' with parentheses
    Main_TFSA_picks = top_picks[(top_picks['Price'] > 20)]

    print(f"\nFound {len(Main_TFSA_picks)} stocks greater than $20.")
    # print(cheap_picks) # Uncomment if you want to see the list in output
else:
    print("DataFrame is empty.")


Found 104 stocks greater than $20.


In [10]:
import pandas as pd
import yfinance as yf
from finvizfinance.quote import finvizfinance
import time
import numpy as np

# --- 1. INPUT YOUR MANUAL LIST HERE ---
MY_TICKERS = ['GRND', 'BTO.TO'] 

def get_combined_watchlist(ticker_list):
    print(f"--- Processing {len(ticker_list)} stocks ---")
    
    # --- PART A: Get Analyst Ratings from Finviz ---
    print("1. Fetching Analyst Ratings from Finviz...")
    finviz_data = []
    
    for ticker in ticker_list:
        try:
            stock = finvizfinance(ticker)
            info = stock.ticker_fundament()
            
            finviz_data.append({
                'Ticker': ticker,
                'Recom': info.get('Recom', np.nan),
                'Target_Price': info.get('Target Price', np.nan)
            })
            time.sleep(0.5) 
            
        except Exception as e:
            print(f"   Skipping Finviz for {ticker}: {e}")
            finviz_data.append({'Ticker': ticker, 'Recom': np.nan, 'Target_Price': np.nan})

    df_finviz = pd.DataFrame(finviz_data)
    
    # --- PART B: Get Real-Time Stats from yfinance ---
    print("2. Fetching Price & Volatility from yfinance...")
    
    try:
        # Download data
        data = yf.download(ticker_list, period="1y", interval="1d", group_by='ticker', progress=False, threads=True)
        yf_stats = []
        
        for ticker in ticker_list:
            try:
                # --- FIXED: Robust Data Extraction ---
                # Check if data has MultiIndex columns (nested structure)
                if isinstance(data.columns, pd.MultiIndex):
                    if ticker in data.columns.levels[0]:
                        df = data[ticker].copy()
                    else:
                        print(f"   Warning: {ticker} not found in yfinance download.")
                        continue
                else:
                    # Single level columns (Flat)
                    df = data.copy()

                # Cleanup
                df = df.dropna(subset=['Close'])
                if len(df) < 20: 
                    print(f"   Warning: Not enough data for {ticker}")
                    continue

                # --- MATH CALCULATIONS ---
                current_price = df['Close'].iloc[-1]
                prev_close = df['Close'].iloc[-2]
                
                high_52 = df['High'].max()
                drop_from_high = ((current_price - high_52) / high_52) * 100
                
                change_pct = ((current_price - prev_close) / prev_close) * 100
                
                # Volatility (30-day Std Dev)
                volatility = df['Close'].pct_change().std() * 100
                
                # Relative Volume
                curr_vol = df['Volume'].iloc[-1]
                avg_vol = df['Volume'].tail(30).mean()
                rel_vol = curr_vol / avg_vol if avg_vol > 0 else 0

                yf_stats.append({
                    'Ticker': ticker,
                    'Price': round(current_price, 2),
                    'Change_%': round(change_pct, 2),
                    'Drop_from_High_%': round(drop_from_high, 2),
                    'Volatility_%': round(volatility, 2),
                    'Rel_Volume': round(rel_vol, 2)
                })
                
            except Exception as e:
                # Print the actual error so we know what's wrong
                print(f"   Error calculating stats for {ticker}: {e}")
                continue
                
        df_yf = pd.DataFrame(yf_stats)
        
    except Exception as e:
        print(f"yfinance Critical Error: {e}")
        return pd.DataFrame()

    # --- PART C: Merge ---
    # We check if df_yf is empty, but we allow df_finviz to have data (Outer Merge)
    if not df_finviz.empty:
        if not df_yf.empty:
            master_df = pd.merge(df_finviz, df_yf, on='Ticker', how='outer')
        else:
            # If yfinance failed completely, just return Finviz data
            print("   Warning: Returning Finviz data only (yfinance failed).")
            master_df = df_finviz
            
        cols = ['Ticker', 'Price', 'Change_%', 'Drop_from_High_%', 'Recom', 'Target_Price', 'Rel_Volume', 'Volatility_%']
        # Filter only columns that exist
        final_cols = [c for c in cols if c in master_df.columns]
        return master_df[final_cols]
    else:
        return pd.DataFrame()

# --- RUN IT ---
watchlist_df = get_combined_watchlist(MY_TICKERS)

if not watchlist_df.empty:
    # Ensure numeric for sorting
    if 'Drop_from_High_%' in watchlist_df.columns:
        watchlist_df['Drop_from_High_%'] = pd.to_numeric(watchlist_df['Drop_from_High_%'], errors='coerce')
        print("\n--- Final Watchlist ---")
        display(watchlist_df.sort_values(by='Drop_from_High_%', ascending=True))
    else:
        display(watchlist_df)
else:
    print("No data found.")

--- Processing 2 stocks ---
1. Fetching Analyst Ratings from Finviz...
   Skipping Finviz for BTO.TO: HTTP error for URL https://finviz.com/quote.ashx?t=BTO.TO: 404 Client Error: Not Found for url: https://finviz.com/quote.ashx?t=BTO.TO
2. Fetching Price & Volatility from yfinance...


C:\Users\jdcc3\AppData\Local\Temp\ipykernel_7392\2137955490.py:40: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker_list, period="1y", interval="1d", group_by='ticker', progress=False, threads=True)



--- Final Watchlist ---


,Ticker,Price,Change_%,Drop_from_High_%,Recom,Target_Price,Rel_Volume,Volatility_%
1,GRND,13.89,1.46,-44.73,1.40,21.75,0.66,3.19
0,BTO.TO,6.31,1.94,-24.10,NaN,NaN,1.64,2.86
